<center> 
    <h1> What cars are Bulgarians buying second hand? </h1>
    <h3>Ivan Svilenov Penchev</h3>
</center>





<center> <h4> Synopsis </h4></center>

---

### Introduction

Cars and the Bulgarian mentality have always been an interesting mix. While, worldwide, countries are lobbing for the use of new cars which are more energy effienct, Bulgaria's faviourite choice for cars seems to be the secondary market. 

> Only for the last year the number of bought and registered new cars was 26 000, while the imported used cars from EU was 266 000. [1]

This of course was the case for 2017, if we compare it to Q1 of 2019 we see an improvement by 29% [2] The number seems substantial, however it is not when we compare the rate of vehicles per capita.


Make a long story short the 9th poorest country in Europe by GDP per capita in international dollars prefers to buy more used cars than new ones.


<div style='float: center'>
  <img style='width: 200px' src="./images/01.png"></img>
</div>

This in itself, while it contributes to the expansion of the biggest in human's history envriomental crisis, is not something disturbing. Notwithstanding, when the lowest among goods markets for three consecutive years, which it also registered the worst performance of all goods markets on problems experienced by consumers and on comparability[3] is the country's first choice for a market a further analysis and monitoring of said market is needed. 

Based on the above mentioned, we compose the two leading areas of interest of this paper:
1. What is the current state of the second hand car market in Bulgaria?
2. Are cars bought in the second hand car market energy effient?

### Data acquisition 
A cornerstone of science is the possibility to critically assess the correctness of scientific claims made and conclusions drawn by other scientists.

In order for this to be possible a systematic approach to and precise description of experimental procedure and subsequent data analysis is needed.

One the first thing we do for enabling reproducibility of results for further analysis and validation by other scientists is descrbing how we aquired our data.

For this paper the primary source of information was the website cars.bg, where a webscraping approach was used, this approach is visually described in the picture bellow, while the full code is included in [in this folder](./webscapper_cars.bg).

<div style='float: center'>
  <img src="./images/02.png"></img>
</div>

The data we receive is not 100% clean and further work on it will be needed.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
from enum import Enum
import datetime
import re

from nose.tools import *

In [ ]:
initial_car_df_from_cars_bg = pd.read_json('./data/json_data.txt', 'r', encoding='utf-8-sig')
#initial_car_df_from_cars_bg.head(10)

In [ ]:
initial_car_df_from_cars_bg = initial_car_df_from_cars_bg.drop("py/object", axis=1)
initial_car_df_from_cars_bg.rename(
    columns={
        "_car_data": "car_data",
        "_car_extra_data": "car_extra_data",
        "_name": "name",
        "_price": "price",
        "_id": "id"
    },
    inplace=True
)


In [ ]:
class Month(Enum):
    Декември = 12
    Ноември = 11
    Октомври = 10
    Септември = 9
    Август = 8
    Юли = 7
    Юни = 6
    Май = 5
    Април = 4
    Март = 3
    Февруари = 2
    Януари = 1

initial_car_df_car_data = pd.io.json.json_normalize(initial_car_df_from_cars_bg['car_data'].values)

def car_data_cleaning_data(chars, varName):
    if type(chars) is str and varName=='Брой врати':
        return chars.split('/')[0]
    if type(chars) is str and varName=='Пробег':
        return int(chars.split(' ')[0].replace(",", ""))
    if type(chars) is str and varName=='Кубатура':
        return int(chars.split(' ')[0].replace(",", ""))
    if type(chars) is str and varName=='Мощност':
        return int(chars.split(' ')[0].replace(",", ""))
    if type(chars) is str and varName=='Година':
        month = Month[chars.split(' ')[0]]
        year = int(chars.split(' ')[1])
        return datetime.datetime(year, month.value, 1)

    
initial_car_df_car_data['Брой врати'] = initial_car_df_car_data.apply(lambda row: car_data_cleaning_data(row['Брой врати'],'Брой врати'), axis=1)
initial_car_df_car_data['Пробег'] = initial_car_df_car_data.apply(lambda row: car_data_cleaning_data(row['Пробег'],'Пробег'), axis=1)
initial_car_df_car_data['Пробег'] = initial_car_df_car_data['Пробег'].fillna(0)
initial_car_df_car_data['Пробег'] = initial_car_df_car_data['Пробег'].astype(int)

initial_car_df_car_data['Кубатура'] = initial_car_df_car_data.apply(lambda row: car_data_cleaning_data(row['Кубатура'],'Кубатура'), axis=1)
initial_car_df_car_data['Кубатура'] = initial_car_df_car_data['Кубатура'].fillna(0)
initial_car_df_car_data['Кубатура'] = initial_car_df_car_data['Кубатура'].astype(int)

initial_car_df_car_data['Мощност'] = initial_car_df_car_data.apply(lambda row: car_data_cleaning_data(row['Мощност'],'Мощност'), axis=1)
initial_car_df_car_data['Мощност'] = initial_car_df_car_data['Мощност'].fillna(0)
initial_car_df_car_data['Мощност'] = initial_car_df_car_data['Мощност'].astype(int)

initial_car_df_car_data['Година'] = initial_car_df_car_data.apply(lambda row: car_data_cleaning_data(row['Година'],'Година'), axis=1)

initial_car_df_car_data.head(4)

In [ ]:
initial_car_df_from_cars_bg = pd.concat([initial_car_df_from_cars_bg, initial_car_df_car_data], axis=1, sort=False)
initial_car_df_from_cars_bg = initial_car_df_from_cars_bg.drop(['car_data'], axis=1)
#initial_car_df_from_cars_bg.head(3)

In [ ]:
all_car_brands_df = pd.read_csv('./data/car_brands.csv', header=None, usecols=[0,0])
all_car_brands_df.columns = ['brand_name']
all_car_brands_df.head()

def my_carbrand(c):
    if type(c) is str:
        first = c.split(' ')
        try:
            brand_row = all_car_brands_df[all_car_brands_df.brand_name==first[0].replace('-','').replace(' ','')]
            valToReturn = brand_row.brand_name.item()
            return valToReturn
        except ValueError:
            try:
                brand_row = all_car_brands_df[all_car_brands_df.brand_name==first[0].strip()+first[1].strip()]
                valToReturn = brand_row.brand_name.item()
                return valToReturn
            except ValueError:
                return pd.np.NaN
        

initial_car_df_from_cars_bg['car_brand'] = initial_car_df_from_cars_bg.apply(lambda row: my_carbrand(row['name']), axis=1)


In [ ]:
def isSold(r):
    httpCon = HttpConnection()
    url = 'https://www.cars.bg/offer/'+r
    r = httpCon.requestSession.get(url)
    if r.headers.get('Set-Cookie') is None:
        print("id " + url + ' is SOLD')
        return True
    else:
        print("id " + url + ' NOT SOLD')
        return False
    
#Sync call, takes fucking forever to finish, Python paralel HOW?!? 
#initial_car_df_from_cars_bg["sold"] = initial_car_df_from_cars_bg['id'].apply(lambda row: isSold(row))
# with open('car_data_with_brand_and_sold_status.json', 'w', encoding='utf-8') as file:
#      df.to_json(file, force_ascii=False, date_format="iso")
print("Can't put your memes in code cell since they are images?")
image = mpimg.imread("images/03.png")
plt.imshow(image)
plt.title("Python and parallel HTTP requests")
plt.axis('off')
plt.show()
image = mpimg.imread("images/04.jpg")
plt.imshow(image)
plt.axis('off')
plt.title("me trying import asyncio and using event_loop()")
plt.show()



In [2]:
initial_car_df_from_cars_bg = pd.read_json('data/car_data_with_brand_and_sold_status.json', 'r', encoding='utf-8-sig')
initial_car_df_from_cars_bg['Година'] = pd.to_datetime(initial_car_df_from_cars_bg['Година'])
initial_car_df_from_cars_bg['Брой врати'] = initial_car_df_from_cars_bg['Брой врати'].fillna(0)
initial_car_df_from_cars_bg['Брой врати'] = initial_car_df_from_cars_bg['Брой врати'].astype(int)
df_car_models = pd.read_csv('Data/car_brands_and_models.csv', header=None, usecols=[1,2])
df_car_models.columns = ['brand_name', 'model_name']

def matchCarOfferNameWithCarModel(name, brand):
    name = name
    brandName= brand
    all_models_list = df_car_models[df_car_models['brand_name'].str.contains(brandName)].model_name.to_list()
    bestMatchModel = ''
    for model in all_models_list:
        for match in re.finditer(model, name):
            s = match.start()
            e = match.end()
            lenght = e-s
            if lenght > len(bestMatchModel):
                bestMatchModel=model
    return pd.np.NaN if len(bestMatchModel)==0 else bestMatchModel
initial_car_df_from_cars_bg['car_model'] = initial_car_df_from_cars_bg[initial_car_df_from_cars_bg['car_brand'].isna()==False].apply(lambda row: matchCarOfferNameWithCarModel(row['name'], row['car_brand']), axis=1)



In [3]:
initial_car_df_car_extra_data = pd.io.json.json_normalize(initial_car_df_from_cars_bg['car_extra_data'].values)
all_posibile_dummy_variable_names = {}
def saveDummyVariableToDictionary(resultList, categoryResultComesFrom, dictionaryToSaveTo):
    for result in resultList:
        if categoryResultComesFrom.lower()+"_"+result.lower().replace(".", " ").replace("  ", " ").replace(" ", "_") not in dictionaryToSaveTo:
             dictionaryToSaveTo[categoryResultComesFrom.lower()+"_"+result.lower().replace(".", " ").replace("  ", " ").replace(" ", "_")]=categoryResultComesFrom.lower()+"_"+result.lower().replace(".", " ").replace("  ", " ").replace(" ", "_")
        
def findAllPossibleDummyVariables(row):
    if type(row['Друго']) is str:
        if len(row['Друго'])>0:
            resultOther = [x.strip() for x in row['Друго'].split(',')]
            saveDummyVariableToDictionary(resultOther, 'Друго',all_posibile_dummy_variable_names)
    if type(row['Комфорт']) is str:        
        if len(row['Комфорт'])>0:
            resultComfort = [x.strip() for x in row['Комфорт'].split(',')]
            saveDummyVariableToDictionary(resultComfort, 'Комфорт',all_posibile_dummy_variable_names)
    if type(row['Сигурност']) is str:        
        if len(row['Сигурност'])>0:
            resultSecurity = [x.strip() for x in row['Сигурност'].split(',')]  
            saveDummyVariableToDictionary(resultSecurity, 'Сигурност',all_posibile_dummy_variable_names)
    
initial_car_df_car_extra_data.apply(lambda row: findAllPossibleDummyVariables(row), axis=1)
print("apply returns a series, which Jupyter displays, this is here to hide it")

apply returns a series, which Jupyter displays, this is here to hide it


In [4]:

def setKeyForIndexTo(dataframe, index, key, value):
    dataframe.at[index,key] = value
    
def populateDummyVariablesForDataFrame(dataframe, dictionaryDummyVariables):
    for key in dictionaryDummyVariables.keys():
        dataframe[key] = False
    
    for i, row in dataframe.iterrows():
        keyToSetForThisRow = []
        if type(row['Друго']) is str and type(row['Друго']) is not float:
            if len(row['Друго'])>0:
                resultOther = [x.strip() for x in row['Друго'].split(',')]
                for result in resultOther:
                    key = "друго"+"_"+result.lower().replace(".", " ").replace("  ", " ").replace(" ", "_")
                    keyToSetForThisRow.append(key)
        if type(row['Комфорт']) is str and type(row['Комфорт']) is not float:
            if len(row['Комфорт'])>0:
                resultComfort = [x.strip() for x in row['Комфорт'].split(',')]
                for result in resultComfort:
                    key = "комфорт"+"_"+result.lower().replace(".", " ").replace("  ", " ").replace(" ", "_")
                    keyToSetForThisRow.append(key)            
        if type(row['Сигурност']) is str and type(row['Сигурност']) is not float:
            if len(row['Сигурност'])>0:
                resultSecurity = [x.strip() for x in row['Сигурност'].split(',')]
                for result in resultSecurity:
                    key = "сигурност"+"_"+result.lower().replace(".", " ").replace("  ", " ").replace(" ", "_")
                    keyToSetForThisRow.append(key)
        if(len(keyToSetForThisRow)>0):
            for key in keyToSetForThisRow:
                setKeyForIndexTo(dataframe,i, key, True)
            
populateDummyVariablesForDataFrame(initial_car_df_car_extra_data,all_posibile_dummy_variable_names) 
initial_car_df_car_extra_data['Евростандарт'] = initial_car_df_car_extra_data['Евростандарт'].map({'EURO 6': 6, 'EURO 5': 5, 'EURO 4': 4, 'EURO 3': 3, 'EURO 2': 2, 'EURO 1': 1, 0:0})
initial_car_df_car_extra_data = initial_car_df_car_extra_data.drop(['Друго','Комфорт','Описание','Сигурност'], axis=1)


    

,Евростандарт,град,частно_лице,друго_автопилот,друго_серво_управление,друго_бордови_компютър,друго_сервизна_книжка,друго_навигационна_система,комфорт_климатроник,комфорт_кожен_салон,...,сигурност_безключово_палене,друго_гаранция,друго_теглич,друго_десен_волан,сигурност_старт-стоп_система,друго_тунинг,сигурност_брониран,друго_ретро,друго_хладилен,друго_taxi
0,5,Русе,False,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,5,Русе,False,True,True,True,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
2,3,Габрово,True,False,True,True,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,5,Бургас,False,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [5]:
initial_car_df_from_cars_bg = pd.concat([initial_car_df_from_cars_bg, initial_car_df_car_extra_data], axis=1, sort=False)
initial_car_df_from_cars_bg = initial_car_df_from_cars_bg.drop(['car_extra_data'], axis=1)

In [92]:
#Човекът е сбъркал модела в описанието
initial_car_df_from_cars_bg.at[57257, 'car_model'] = 'K2700'
#Човекът е писал като модел "всички"
initial_car_df_from_cars_bg.at[24860, 'car_model'] = '2'
#Човекът е писал в описанието Mecedes Benz, a е имал предвид субару
initial_car_df_from_cars_bg.at[25085, 'car_model'] = 'Impreza'
initial_car_df_from_cars_bg.at[25085, 'car_brand'] = 'Subaru'
#Не засича, заради +2?
initial_car_df_from_cars_bg.at[122, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[11317, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[23612, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[24193, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[24865, 'car_model'] = 'X-Trail'
initial_car_df_from_cars_bg.at[30938, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[39948, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[40151, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[45074, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[45124, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[52217, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[52649, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[53621, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[55451, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[58082, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[70996, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[72357, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[73743, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[76857, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[76948, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[78863, 'car_model'] = 'Qashqai+2'
#
initial_car_df_from_cars_bg.at[78564, 'car_model'] = 'Golf'
initial_car_df_from_cars_bg.at[39299, 'car_model'] = '13'
#Range rover
initial_car_df_from_cars_bg.at[31416, 'car_model'] = 'P38'
initial_car_df_from_cars_bg.at[26041, 'car_model'] = 'Sport'
initial_car_df_from_cars_bg.at[24826, 'car_model'] = 'Vogue'
initial_car_df_from_cars_bg.at[56465, 'car_model'] = 'Vogue'
initial_car_df_from_cars_bg.at[85213, 'car_model'] = 'Vogue'
initial_car_df_from_cars_bg.at[37829, 'car_model'] = 'Vogue'
initial_car_df_from_cars_bg.at[38367, 'car_model'] = 'Vogue'
initial_car_df_from_cars_bg = initial_car_df_from_cars_bg[pd.notnull(initial_car_df_from_cars_bg['car_model'])]

DieselAbbr = ['TDI','CDTI','dCi']                
for i, row in initial_car_df_from_cars_bg[(initial_car_df_from_cars_bg['Гориво'].isna()==True) & (initial_car_df_from_cars_bg['name'].str.contains('|'.join(DieselAbbr), flags=re.I)==True)].iterrows():
    setKeyForIndexTo(initial_car_df_from_cars_bg,i, 'Гориво', 'Дизел')
for i, row in initial_car_df_from_cars_bg[(initial_car_df_from_cars_bg['Гориво'].isna()==True)].iterrows():
    setKeyForIndexTo(initial_car_df_from_cars_bg,i, 'Гориво', 'Бензин')    

,id,name,price,Брой врати,Година,Гориво,Кубатура,Мощност,Пробег,Скорости,...,сигурност_безключово_палене,друго_гаранция,друго_теглич,друго_десен_волан,сигурност_старт-стоп_система,друго_тунинг,сигурност_брониран,друго_ретро,друго_хладилен,друго_taxi
0,c3126538,Skoda Superb NAV+DVD+KAM+6sp,16900,4,2010-12-01 00:00:00+00:00,Дизел,2000,140,148291,Ръчни,...,False,False,False,False,False,False,False,False,False,False
1,c3137948,VW Golf HIGH-LINE+PEARL,11400,4,2009-12-01 00:00:00+00:00,Дизел,2000,110,186509,Ръчни,...,False,False,False,False,False,False,False,False,False,False
2,c3121884,Audi A3 110kc,2560,4,2001-06-01 00:00:00+00:00,Дизел,0,110,210000,Ръчни,...,False,False,False,False,False,False,False,False,False,False
3,c3044418,Ford C-Max 1.8TDCI,3999,4,2005-09-01 00:00:00+00:00,Дизел,1800,136,197000,Ръчни,...,False,False,False,False,False,False,False,False,False,False
4,c3071808,VW Sharan 1.9TDI 116к.с,5500,4,2003-08-01 00:00:00+00:00,Дизел,1900,116,223000,Ръчни,...,False,False,False,False,False,False,False,False,False,False
5,c2994883,Audi A4 1.9TDi 131к.с,5800,4,2003-09-01 00:00:00+00:00,Дизел,1900,131,193000,Ръчни,...,False,False,False,False,False,False,False,False,False,False
6,c3119603,Nissan 100 NX 1.6,1500,2,1993-08-01 00:00:00+00:00,Газ/Бензин,1600,90,220000,Ръчни,...,False,False,False,False,False,False,False,False,False,False
7,c2928548,Mitsubishi Space Star 1.8GDI,698,4,1999-05-01 00:00:00+00:00,Бензин,1800,122,245000,Ръчни,...,False,False,False,False,False,False,False,False,False,False
8,c2846469,Renault Clio 1.5DCI,2999,4,2005-09-01 00:00:00+00:00,Дизел,1500,82,189000,Ръчни,...,False,False,False,False,False,False,False,False,False,False
9,c3015498,VW Polo 1.4i GOAL,5200,4,2006-09-01 00:00:00+00:00,Бензин,1400,80,205000,Ръчни,...,False,False,False,False,False,False,False,False,False,False


In [93]:
initial_car_df_from_cars_bg['Гориво'].unique()

array(['Дизел', 'Газ/Бензин', 'Бензин', 'Метан/Бензин', 'Електричество',
       'Хибрид'], dtype=object)

### References
1. [В България се карат най-старите коли в ЕС](https://www.investor.bg/analizi/447/a/v-bylgariia-se-karat-nai-starite-koli-v-es-248385/)
2. [Статистика: 10 пъти по-бедни сме от германците по покупки на нови коли](https://www.dnes.bg/obshtestvo/2019/05/18/statistika-10-pyti-po-bedni-sme-ot-germancite-po-novi-koli.410667)
3. [Study on the second hand cars market](https://ec.europa.eu/info/publications/study-second-hand-cars-market_en)
4. [European second-hand car market analysis](https://www.oeko.de/oekodoc/1114/2011-005-en.pdf)
